In [1]:
import premise as ps

In [2]:
import bw2io as bi
import bw2data as bd

In [3]:
bd.projects.set_current('MCC')

In [4]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [5]:
ei391cdir = "/home/haithamth/Documents/ecoinvent/ecoin_cuttoff_391/ecoinvent 3.9.1_cutoff_ecoSpold02/datasets"
data_base_name = "ecoinvent-3.9.1-cuttoff"
if data_base_name in bd.databases:
    print("Database has already been imported")
else:
    ei391c = bi.SingleOutputEcospold2Importer(ei391cdir, data_base_name)
    ei391c.apply_strategies()
    ei391c.statistics()

    #ei39.drop_unlinked(True)
    ei391c.write_database()

Database has already been imported


In [6]:
from premise import NewDatabase

In [7]:
# ndb = NewDatabase(
#     scenarios=[
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2020},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2020},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2030},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2030},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2040},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
#     ],
#     source_db="ecoinvent-3.9.1-cuttoff", # <-- name of the database in the BW2 project. Must be a string.
#     source_version="3.9.1", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
#     key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=', # <-- decryption key
#     # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
#     use_multiprocessing=True # <-- set to True if you want to use multiprocessing
# )

In [8]:
# ndb.update_electricity()

In [9]:
# ndb.write_db_to_brightway(["SSP2-RCP19_2020","SSP2-RCP26_2020", "SSP2-RCP19_2030","SSP2-RCP26_2030", "SSP2-RCP19_2040","SSP2-RCP26_2040"])

In [10]:
ecn = bd.Database("ecoinvent-3.9.1-cuttoff")

In [11]:
ecn.search('steel unalloyed')[0].key

('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7')

In [12]:
import uuid
class Product:
    def __init__(self, name: str, amount: float | int, unit: str, service_life: int, production_lci: tuple[str,str]):
        # randomly generated id
        self.id = uuid.uuid4().hex
        self.name = name
        self.amount = amount
        self.unit = unit
        self.production_lci = production_lci
        self.service_life = service_life
        self.production_act = bd.get_activity(self.production_lci)

    def __repr__(self):
        return f"{(self.name, self.id)}"
    
    def replacement_sn(self):
        ...

In [13]:
p = Product(name='steel_beam', amount=1, unit='kg', service_life=20, production_lci=('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7'))

In [14]:
p2 = Product(name='steel_beam2', amount=2, unit='kg', service_life=33, production_lci=('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7'))

In [15]:
class Mfa:
    def __init__(self, start: int = 2020, end: int = 2080, time_step: int = 10):
        self.start = start
        self.end = end
        self.time_step = time_step
        self.time_line = list(range(self.start, self.end+1, self.time_step))
        self.points = []
    
    def add_points(self, product: Product):
        first_repl = product.service_life + self.start
        
        for i in range(first_repl, self.end+1, product.service_life):
            self.points.append((i, product))
        # clean duplicates
        self.points = list(set(self.points))
        # sort by year
        self.points = sorted(self.points, key=lambda x: x[0])
        return self.points

In [16]:
tl = Mfa(start=2020, end=2080)

In [17]:
tl.add_points(p)

[(2040, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2060, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2080, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f'))]

In [18]:
tl.add_points(p2)

[(2040, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2053, ('steel_beam2', '121e4bd9497342d6a0073455d31add66')),
 (2060, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2080, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f'))]

In [19]:
tl.points

[(2040, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2053, ('steel_beam2', '121e4bd9497342d6a0073455d31add66')),
 (2060, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f')),
 (2080, ('steel_beam', '4e130608bc004f08b07b8a62a63bc25f'))]

In [20]:
import bw2calc as bc
class ProLCA:
    def __init__(self, product: Product, time_line: Mfa, method: tuple[str,str,str] = ):
        self.product = product
        self.time_line = time_line
    
    def production_lca(self):
        # get the activity from the database
        # create a functional unit
        act = self.product.production_act
        amount = self.product.amount
        fu = {act: amount}
        method = self.method
        lca = bc.LCA(fu, method)
        lca.lci()
        lca.lcia()
        return lca.score
    
    def repl_lca(self):
        # replacement lcas
        # get the timeline
        time_line = self.time_line.points
        

In [21]:
def db_chooser(year: int):
    # choose the database based on the year
    if year < 2030:
        return "SSP2-RCP19_2020"
    elif year < 2040:
        return "SSP2-RCP19_2030"

In [24]:
import premise

In [22]:
def activity_mapper(activity):
    ...

In [41]:
db1 = bd.Database("SSP2-RCP19_2020")
db2 = bd.Database("SSP2-RCP19_2030")
db_eco = bd.Database("ecoinvent-3.9.1-cuttoff")

In [42]:
stl1= db_eco.get("d872e0d78319cb13e12b96de83e19dd7")

In [43]:
stl2 = db2.get("d872e0d78319cb13e12b96de83e19dd7")

In [46]:
# run lca on stl1 and stl2
methd = ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')

stl1_lca = bc.LCA({stl1: 1}, methd)

stl1_lca.lci()

stl1_lca.lcia()

stl1_lca.score

stl2_lca = bc.LCA({stl2: 1}, methd)

stl2_lca.lci()

stl2_lca.lcia()

stl2_lca.score

stl1_lca.score - stl2_lca.score


0.1731943739832147